In [41]:
import pandas as pd
import plotly.graph_objects as go

# Load data
df = pd.read_excel("../data/dados_brutos.xlsx")

# Convert 'Data' column
df['Data'] = pd.to_datetime(df['Data'], format="%Y%m%d", errors='coerce')
df['Ano'] = df['Data'].dt.year

# Process 'Consumo' column
df['Consumo'] = pd.to_numeric(df['Consumo'], errors='coerce')
df.dropna(subset=['Consumo'], inplace=True)

# Create DataFrame with main columns
df_consumo = df[['Ano', 'UF', 'Consumo']].copy()

# Region dictionary
regioes = {
    "Norte": ['RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO'],
    "Nordeste": ['MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE', 'BA'],
    "Sudeste": ['SP', 'RJ', 'MG', 'ES'],
    "Sul": ['PR', 'SC', 'RS'],
    "Centro-Oeste": ['DF', 'GO', 'MT', 'MS']
}

# Group by state and year
consumo_uf_ano = df_consumo.groupby(['UF', 'Ano'])['Consumo'].sum().reset_index()

# Group by region and year
lista_regioes = []
for regiao, estados in regioes.items():
    df_regiao = df_consumo[df_consumo['UF'].isin(estados)]
    consumo_regiao = df_regiao.groupby('Ano')['Consumo'].sum().reset_index()
    consumo_regiao['Regiao'] = regiao
    lista_regioes.append(consumo_regiao)
df_regioes = pd.concat(lista_regioes)

# Top consuming states (total sum)
top_estados = consumo_uf_ano.groupby('UF')['Consumo'].sum().nlargest(5).index.tolist()

# ----------- INTERACTIVE PLOT --------------

fig = go.Figure()

# All states
ufs = consumo_uf_ano['UF'].unique()
for uf in sorted(ufs):
    dados = consumo_uf_ano[consumo_uf_ano['UF'] == uf]
    fig.add_trace(go.Scatter(x=dados['Ano'], y=dados['Consumo'], name=uf, visible=True))

# Regions
for regiao in regioes.keys():
    dados = df_regioes[df_regioes['Regiao'] == regiao]
    fig.add_trace(go.Scatter(x=dados['Ano'], y=dados['Consumo'], name=regiao, visible=False))

# Top consumers
for uf in top_estados:
    dados = consumo_uf_ano[consumo_uf_ano['UF'] == uf]
    fig.add_trace(go.Scatter(x=dados['Ano'], y=dados['Consumo'], name=f"Top: {uf}", visible=False))

# Dropdown Menu
num_ufs = len(ufs)
menu_buttons = [
    {
        'label': 'Todos os Estados',
        'method': 'update',
        'args': [{'visible': [i < num_ufs for i in range(len(fig.data))]},
                {'title': 'Consumo por Estado - Todos'}]
    },
    {
        'label': 'Por Região',
        'method': 'update',
        'args': [{'visible': [num_ufs <= i < num_ufs + len(regioes) for i in range(len(fig.data))]},
                {'title': 'Consumo por Região'}]
    },
    {
        'label': 'Top 5 Estados Consumidores',
        'method': 'update',
        'args': [{'visible': [i >= num_ufs + len(regioes) for i in range(len(fig.data))]},
                {'title': 'Top 5 Estados Consumidores'}]
    }
]

fig.update_layout(
    title="Consumo de Energia por UF ou Região",
    xaxis_title="Ano",
    yaxis_title="Consumo (kWh)",
    updatemenus=[{
        "buttons": menu_buttons,
        "direction": "down",
        "showactive": True,
        "x": 0.0,
        "xanchor": "left",
        "y": 1.15,
        "yanchor": "top"
    }],
    legend_title="UF / Região",
    template="plotly_white",
    height=600,
    width=1000
)

fig.show()
